# RAPPOR – Server-side

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np

Disabling scientific notation makes it a bit easier to compare numeric values with the ones logged in the R part:

In [2]:
np.set_printoptions(suppress=True)

## Read in data files

In production, the content of these files is coming from the clients. For testing, we just use files that are generated by Google's simulations.

In [3]:
base = "../rappor/"
base += "_tmp/python/r-gauss-small-sim_bloom_filter1_1/"

PARAMS_PATH = base + "case_params.csv"
COUNTS_PATH = base + "1/case_counts.csv"
MAP_PATH = base + "case_map.csv"
CANDIDATES_PATH = base + "case_unique_values.txt" #"case_candidates.txt"

### params

In [4]:
k, h, m, p, q, f = pd.read_csv(PARAMS_PATH).iloc[0]
k = int(k)
h = int(h)
m = int(m)

In [5]:
print "Bloom filter has k=%d bits and uses h=%d hash functions" % (k, h)
print "There are m=%d cohorts" % m
print "There is a f=%.2f probability that bits in the Bloom filter are randomly changed" % f
print "p=%.2f, q=%.2f" % (p, q)

Bloom filter has k=4 bits and uses h=2 hash functions
There are m=32 cohorts
There is a f=0.00 probability that bits in the Bloom filter are randomly changed
p=0.50, q=0.75


### counts

Each row contains information about one cohort. The very first value in a row specifies the total number of reports in that cohort. The other values specify how often the respective bit was set in the sent Bloom Filter. Because some bits were randomly changed, the first value can be smaller or greater than the sum of the other values in that row.

In the real server implementation, we will need a little bit of additional logic for calculating these sums. Here, we just use what Google's implementation already provides.

In [6]:
counts = DataFrame.from_csv(COUNTS_PATH, header=None, index_col=None)
counts.head()

,0,1,2,3,4
0,31250,18141,19440,18535,19540
1,31250,18492,19162,19051,18492
2,31250,18769,18670,19149,19340
3,31250,17985,19548,18788,19165
4,31250,18793,19899,19237,18827


**TODO**: Exclude cohorts with $0$ reports

### maps

One row for every candidate string. The leftmost value in a row shows the respective string, the other values (number of cohorts * number of hash functions) show the hashed values of that string

In [7]:
maps = DataFrame.from_csv(MAP_PATH, header=None)
maps.head()

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,,,,,,,,,,,,,,,,,,,,,
v1,2,4,6,7,12,9,15,15,20,17,...,112,109,114,115,119,120,123,121,127,126
v2,3,4,8,7,9,10,16,16,20,18,...,110,110,113,114,120,117,122,122,128,126
v3,2,1,6,5,10,12,16,16,19,19,...,110,109,113,115,120,119,122,123,128,127
v4,2,1,8,6,10,11,14,14,20,19,...,112,110,114,113,120,117,123,123,128,127
v5,1,3,5,7,12,11,16,13,19,19,...,112,111,116,114,120,118,121,122,126,127


### candidates

Here, the candidates are simply the values that were also used by the client-side code. Generally, we would like to use a superset of these values, e.g. the most popular websites.

In [8]:
def get_candidates():
    with open(CANDIDATES_PATH) as f:
        lines = f.readlines()
        return [candidate.strip() for candidate in lines]

In [9]:
candidates = get_candidates()
M = len(candidates)
candidates[:10]

['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10']

## Preparing the data to separate signal and noise

The high-level task of the server-side is to find out which candidate strings were really used by the clients. We will use statistical techniques for doing this, namely linear regression to find out which candidate strings probably influenced the final result.

Another way to think about this: In Machine Learning and statistics we are often concerned with separating signal from noise. Here, the signal is given by the hashed values of candidate strings. The noise is added on purpose by clients in order to maintain privacy. On the server-side, we then try to use statistical techniques to remove the noise from the aggregated data.

### Target values $y$: Estimating true counts of the original Bloom filter

First of all, we do some basic preprocessing to be able to use the same variables that are also being used by the RAPPOR paper. This makes it a bit easier to implement the math formulas given in the paper.

$N$ is a vector containing the number of reports from the individual cohorts. $c$ is a matrix
where $c_{ij}$ tells us how often bit $j$ was set in cohort $i$.

In [10]:
N = counts[0].as_matrix()
c = counts.drop([0], axis=1).as_matrix().T

The target values $y$ will contain estimates of how often the individual bits were really set in the original bloom filter.

$$
t_{ij} = \frac{c_{ij} - (p + 0.5fq - 0.5fp) N_j}{(1 - f) (q - p)}
$$

$y$ is then simply a long vector that the rows of $t$ flattened. `estimate_bloom_counts` calculates $y$ in a vectorized way.

**TODO**: Motivate this formula

In [11]:
def estimate_bloom_count(c, N):
    Y = c - ((p + 0.5 * f * q - 0.5 * f * p) * N)
    Y /= ((1 - f) * (q - p))
    return Y

In [12]:
Y = estimate_bloom_count(c, N)

In [13]:
print Y.shape
assert(Y.shape == (k, m))

(4, 32)


This is not mentioned in the paper, but actually $Y$ is then also divided by $N$ to get frequencies instead of counts. We also reshape the matrix to one long vector, and call this $y$.

In [14]:
y = (Y / N).T.reshape(k * m)

### Design matrix $X$: 

$X$ will have dimensions $km \times M$ where
- $k$ is the number of bits in the Bloom filter
- $m$ is the number of cohorts
- $M$ is the number of candidate strings

Each candidate string corresponds to one feature. Each feature has $h * m$ values set to $1$, all others are $0$. This makes $X$ a sparse matrix.

Each data point corresponds to one bit in a cohort. A cell is set to $1$ if the respective bit in the respective cohort is set using the respective hashed candidate string.

---

To get the bits set in a Bloom filter, we import a function from Google's repo. This is important because we need to make sure to use the same hash function in client and server.

In [15]:
from client.rappor import get_bloom_bits

This function gives us the bits that are set when the candidate string is hashed

In [16]:
get_bloom_bits("test", 4, h, k)

[3, 0]

Next, we can finally create $X$ by creating the matrices of the individual cohorts, and then stacking them vertically.

In [17]:
matrix = []

for cohort in range(m):
    rows = []
    
    for candidate in candidates:
        bits = np.zeros(k)
        bits_set = get_bloom_bits(candidate, cohort, h, k)
        bits[bits_set] = 1
        rows.append(bits)
        
    for row in np.array(rows).T:
        matrix.append(row)

X = np.array(matrix)

In [18]:
print X.shape
assert(X.shape == (k * m, M))

(128, 100)


To give some intuition on what we're doing here: The design matrix $X$ contains information about what bits would be set by which candidates and by using the estimated counts of the individual bits, we try to infer which candidate strings really appeared.

**TODO**: Why statistical techniques? On a high-level view, removing the noise makes sense, but this also seems a bit like a combinatorical problem where you try to find candidates that cover all set bits

**TODO**: Why do we use the estimated results?

## Lasso: Eliminating some candidates

In the RAPPOR paper, Google says they fit a Lasso model and only continue working with candidate strings that have non-zero coefficients. This acts as a preliminary filter to make the next model simpler.

**TODO**: Understand how exactly this helps

However, actually, Google only performs this Lasso regression if the number of candidates $M$ is larger than $0.8 * m * k$, as this means the system is close to being underdetermined.

**TODO**: Understand why this exact check exists

In [19]:
from sklearn.linear_model import Lasso

In [20]:
cut_off = 0.8 * m * k
perform_lasso = M > cut_off

print M, cut_off, "| Lasso should be performed:", perform_lasso

100 102.4 | Lasso should be performed: False


In [21]:
clf = Lasso()
clf.fit(X, y)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [22]:
clf.coef_

array([ 0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,
        0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
        0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,
       -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.])

In [23]:
good_candidates = np.where(clf.coef_ > 0)[0]
good_candidates

array([], dtype=int64)

In [24]:
[candidates[i] for i in good_candidates]

[]

## Calculating standard deviations

Because the estimated $y$ values are just expected values, it's useful to also take spread into account. We do this by calculating the standard deviations of our estimates.

**TODO**: Understand how to arrive at this formula

The formulas for this are not given in the paper, so the code below is directly adapted from the R code:

In [25]:
def _std_row(row, p01, p11, p2):
    N = row[0]
    v = row[1:]
    
    p_hats = (v - p01 * N) / (N * p2) # expectation of a true 1
    p_hats = np.maximum(0, np.minimum(1, p_hats))
    
    r = p_hats * p11 + (1 - p_hats) * p01
        
    return N * r * (1 - r) / p2**2
    
def calculate_variances(counts):
    p01 = p * (1 - f / 2) + q * f / 2
    p11 = q * (1 - f / 2) + p * f / 2
    p2 = p11 - p01
    
    N = counts[0].as_matrix()
    
    return (np.sqrt(counts.apply(lambda row: _std_row(row, p01, p11, p2), axis=1).as_matrix().T) / N).T

In [26]:
stds = calculate_variances(counts).reshape((k * m))

Later, we will use the standard deviations to resample the estimates.

## Updating the data matrix and target values

I'm not entirely sure why this is needed, and it's not mentioned in the paper, but Google also normalizes $X$ and $y$ to new variables $A, b$.

**TODO**: Understand why exactly this is needed

In [27]:
def calculate_normalization(stds):
    w = 1 / stds
    w_median = np.median(w[np.isfinite(w)])

    if not np.isfinite(w_median):
        w_median = 1

    w = np.minimum(w, 2 * w_median)
    w = w / w.mean()

    return w

In [28]:
def normalize(X, y, stds):
    w = calculate_normalization(stds)
    
    A = np.diag(w).dot(X)
    b = y * w
    
    return A, b

In [29]:
A, b = normalize(X, y, stds)

## Second Linear Regression

In the paper, Google says they perform a "regular least-squares regression using the selected variables" but this is not true. The linear regression [involves additional constraints](https://github.com/google/rappor/blob/master/analysis/R/alternative.R#L22-26) (that are not described in the paper), and the input variables are also changed a bit, as we saw before.

The coefficients $x$ must follow three constraints:

- $x$ must be nonnegative
- $x$ summed up to a value smaller or equal to $1$
- The individual predictions using $x$ must not exceed the estimates by 3 standard deviations

**TODO**: Understand the motivation behind these constraints

The first constraint is pretty basic and can be enforced by a simple bound. The second constraint is automatically enforced by most implementations, and is also easy to enforce manually for linear models after fitting.

The third constraint however makes things a bit more difficult. In general, sklearn and SciPy cannot deal with this automatically. Because of this, we have to fallback to a more low-level optimizer here that allows us to encode arbitrary constraints.

In [30]:
from scipy.optimize import minimize, nnls
from numpy.linalg import norm

For the third constraint, we introduce a new variable $yy$ that gives a limit for how much predicted values may differ.

In [31]:
yy = np.minimum(1, np.maximum(y + 3 * stds, 0.01))

Because of the generality of our optimizer, it's important to choose a good first guess $x_0$. We generate this guess by quickly training a model using the first two constraints, but without the third. This turns out to be the critical change that allows us to get the same results as the optimization library used in R by Google.

In [32]:
x0, _ = nnls(A, b)

The cost function is simply least squares, i.e. we want to minimize $||Ax - b||_2^2$.

In [33]:
def cost(x):
    return norm(A.dot(x) - b, ord=2)**2

Finally, we encode the third constraint, and run the minimizer.

In [34]:
standard_deviation_constraint = {
    "type": "ineq",
    "fun": lambda x: max(X.dot(x) - yy)
}

In [35]:
x = minimize(cost,
         x0=x0,
         method="TNC",
         constraints=standard_deviation_constraint,
         bounds=zip(np.zeros(M), np.ones(M))
        ).x

/usr/local/lib/python2.7/site-packages/scipy/optimize/_minimize.py:397: RuntimeWarning: Method TNC cannot handle constraints.
  RuntimeWarning)


It depends a bit on the dataset, but usually we get the exact same results as Google's RAPPOR here.

In [36]:
x

array([ 0.        ,  0.00182652,  0.00012597,  0.00424327,  0.00158191,
        0.        ,  0.        ,  0.        ,  0.00972536,  0.        ,
        0.        ,  0.0049677 ,  0.00074506,  0.00263669,  0.00111928,
        0.00012783,  0.00300057,  0.00013777,  0.01000537,  0.00707541,
        0.01147606,  0.0083258 ,  0.01341316,  0.00789701,  0.00355234,
        0.00697167,  0.00975315,  0.01727338,  0.01124086,  0.01484703,
        0.01612965,  0.01190803,  0.0132142 ,  0.01554626,  0.01074183,
        0.01487565,  0.01876687,  0.02007674,  0.01784912,  0.02188738,
        0.02326081,  0.02326737,  0.02325578,  0.02368823,  0.02070788,
        0.01840034,  0.02729136,  0.015493  ,  0.02097782,  0.02094391,
        0.02189653,  0.02035882,  0.02031533,  0.02121134,  0.02329346,
        0.02112202,  0.02205307,  0.02972841,  0.02478986,  0.01782075,
        0.01936563,  0.01602125,  0.01628776,  0.0200774 ,  0.02004624,
        0.01567713,  0.01872445,  0.01324706,  0.01072458,  0.01

In [37]:
np.set_printoptions(suppress=False)
x

array([ 0.        ,  0.00182652,  0.00012597,  0.00424327,  0.00158191,
        0.        ,  0.        ,  0.        ,  0.00972536,  0.        ,
        0.        ,  0.0049677 ,  0.00074506,  0.00263669,  0.00111928,
        0.00012783,  0.00300057,  0.00013777,  0.01000537,  0.00707541,
        0.01147606,  0.0083258 ,  0.01341316,  0.00789701,  0.00355234,
        0.00697167,  0.00975315,  0.01727338,  0.01124086,  0.01484703,
        0.01612965,  0.01190803,  0.0132142 ,  0.01554626,  0.01074183,
        0.01487565,  0.01876687,  0.02007674,  0.01784912,  0.02188738,
        0.02326081,  0.02326737,  0.02325578,  0.02368823,  0.02070788,
        0.01840034,  0.02729136,  0.015493  ,  0.02097782,  0.02094391,
        0.02189653,  0.02035882,  0.02031533,  0.02121134,  0.02329346,
        0.02112202,  0.02205307,  0.02972841,  0.02478986,  0.01782075,
        0.01936563,  0.01602125,  0.01628776,  0.0200774 ,  0.02004624,
        0.01567713,  0.01872445,  0.01324706,  0.01072458,  0.01

Finally, we can put it all together:

In [38]:
def fit(X, y, stds):
    A, b = normalize(X, y, stds)
    yy = np.minimum(1, np.maximum(y + 3 * stds, 0.01))
    
    x0, _ = nnls(A, b)
    x = minimize(cost,
         x0=x0,
         method="TNC",
         constraints=standard_deviation_constraint,
         bounds=zip(np.zeros(M), np.ones(M))
        ).x
    
    return x

In [39]:
fit(X, y, stds)

array([ 0.        ,  0.00182652,  0.00012597,  0.00424327,  0.00158191,
        0.        ,  0.        ,  0.        ,  0.00972536,  0.        ,
        0.        ,  0.0049677 ,  0.00074506,  0.00263669,  0.00111928,
        0.00012783,  0.00300057,  0.00013777,  0.01000537,  0.00707541,
        0.01147606,  0.0083258 ,  0.01341316,  0.00789701,  0.00355234,
        0.00697167,  0.00975315,  0.01727338,  0.01124086,  0.01484703,
        0.01612965,  0.01190803,  0.0132142 ,  0.01554626,  0.01074183,
        0.01487565,  0.01876687,  0.02007674,  0.01784912,  0.02188738,
        0.02326081,  0.02326737,  0.02325578,  0.02368823,  0.02070788,
        0.01840034,  0.02729136,  0.015493  ,  0.02097782,  0.02094391,
        0.02189653,  0.02035882,  0.02031533,  0.02121134,  0.02329346,
        0.02112202,  0.02205307,  0.02972841,  0.02478986,  0.01782075,
        0.01936563,  0.01602125,  0.01628776,  0.0200774 ,  0.02004624,
        0.01567713,  0.01872445,  0.01324706,  0.01072458,  0.01

## Resampling

A popular way to get more data is resampling. Here, we use the standard deviations to do this.

In [47]:
from numpy.random import normal

In [48]:
def resample(y, stds):
    deviation = np.array([normal(0, std) for std in stds])
    y += deviation
    
    stds = np.sqrt(stds)
    
    return y, stds

In [49]:
coefs = []

for i in range(5):
    if i > 0:
        y_resampled, stds_resampled = resample(y, stds)
    else:
        y_resampled, stds_resampled = y, stds
        
    coefs.append(fit(X, y_resampled, stds_resampled))

In [50]:
coefs

[array([  0.00000000e+00,   1.70933534e-03,   3.74906109e-06,
          4.29501743e-03,   1.73843614e-03,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   9.58005658e-03,
          0.00000000e+00,   0.00000000e+00,   5.09126606e-03,
          5.77950980e-04,   2.64463079e-03,   1.01352657e-03,
          8.63807391e-05,   3.03610763e-03,   0.00000000e+00,
          9.78693776e-03,   6.88427641e-03,   1.14197607e-02,
          8.15443339e-03,   1.36308376e-02,   7.64156313e-03,
          3.40476071e-03,   6.91822261e-03,   9.81677476e-03,
          1.73191792e-02,   1.14651887e-02,   1.48792707e-02,
          1.62943244e-02,   1.18360884e-02,   1.33135127e-02,
          1.56252280e-02,   1.07126563e-02,   1.46330569e-02,
          1.89323380e-02,   2.01149103e-02,   1.80911209e-02,
          2.18048915e-02,   2.34755046e-02,   2.33809697e-02,
          2.34450964e-02,   2.36142097e-02,   2.07317091e-02,
          1.82266311e-02,   2.74309187e-02,   1.54997230e-02,
        

## Statistical tests

In [44]:
from sklearn.feature_selection import f_regression
from statsmodels.sandbox.stats.multicomp import multipletests

In [45]:
# F, ps = f_regression(X_good, y_good)
# ps_corrected = multipletests(ps, method='bonferroni')
# ps_corrected

In [46]:
# F.sort()
# F